In [1]:
import pandas as pd

### We manually test all pipelines for each similarity threshold (from 0.1-0.9) and save them respectively

In [ ]:
RAGv1 = pd.read_csv('sythetic_testing/QB1_threshold_0.1.csv')
RAGv2 = pd.read_csv('sythetic_testing/QB2_threshold_0.1.csv')
RAGv3 = pd.read_csv('sythetic_testing/QB3_threshold_0.1.csv')
RAGv4 = pd.read_csv('sythetic_testing/QB4_threshold_0.1.csv')
RAGv5 = pd.read_csv('sythetic_testing/HB1_0.1.csv')
RAGv6 = pd.read_csv('sythetic_testing/HB2_0.1.csv')
RAGv7 = pd.read_csv('sythetic_testing/HYB_0.1.csv')
# baseline = pd.read_csv('sythetic_testing/baseline.csv')

In [7]:
from bs4 import BeautifulSoup

def clean_accepted_answer_column(df):
    """
    Cleans the 'Accepted Answer' column by removing HTML tags and keeping only the text.
    """
    df['Accepted Answer Cleaned'] = df['Accepted Answer'].apply(
        lambda x: BeautifulSoup(str(x), 'html.parser').get_text().strip()
    )
    return df

In [8]:
RAGv3 = clean_accepted_answer_column(RAGv3)

In [12]:
import pandas as pd

def fix_if_unseen_by_context(df, col='Step3PROMPT'):
    
    def check_context(row):
        # If IFUnseen is already 0, do not apply changes
        if row['IFUnseen'] == 0:
            return 0
        
        text = str(row[col])
        
        # Define markers
        marker_start = '### CONTEXT:'
        marker_end = 'Please provide your best answer below:'
        
        # Find the start index
        idx_start = text.find(marker_start)
        
        # If '### CONTEXT:' not found at all, we assume there's some content
        if idx_start == -1:
            return 1
        
        # Extract everything after '### CONTEXT:'
        context_part = text[idx_start + len(marker_start):]
        
        # Find the end marker
        idx_end = context_part.find(marker_end)
        
        # If we do find 'Please provide your best answer below:', cut off at that point
        if idx_end != -1:
            context_part = context_part[:idx_end]
        
        # Trim whitespace
        context_part = context_part.strip()

        # Condition 2: Contains the phrase "no relevant"
        if not context_part or "no relevant" in context_part.lower():
            return 0
        
        # Otherwise, we say it has content
        return 1
    
    # Apply our check_context logic row by row
    df['IFUnseen'] = df.apply(check_context, axis=1)
    
    return df




In [13]:
RAGv3 = fix_if_unseen_by_context(RAGv3, col='Step3PROMPT')

In [14]:
RAGv3['IFUnseen'].value_counts()

IFUnseen
0    385
Name: count, dtype: int64

In [15]:
def extract_answer(text):
    if isinstance(text, str) and "Please provide your best answer below:" in text:
        return text.split("Please provide your best answer below:")[-1].strip()
    return text

# Process the Step2Response column
RAGv1['Step2Response'] = RAGv1['Step2Response'].apply(extract_answer)

In [16]:
RAGv2['Step2Response_v2'] = RAGv2['Step2Response_v2'].apply(extract_answer)

In [17]:
import re

def is_context_empty(cell_text):
    """
    Returns True if the text between '### CONTEXT:' and 
    'Please provide your best answer below:' is:
      1) Empty (after trimming whitespace), OR
      2) Exactly '[]' (with optional whitespace inside the brackets), OR
      3) Contains 'no relevant' (case-insensitive).

    Returns False otherwise, or if '### CONTEXT:' is not found.
    """
    # Convert to string to avoid errors with non-string data
    text = str(cell_text)

    marker_start = "### CONTEXT:"
    marker_end = "Please provide your best answer below:"

    # Case-insensitive search
    text_lower = text.lower()
    start_lower = marker_start.lower()
    end_lower = marker_end.lower()

    # Find the start index of '### CONTEXT:' (case-insensitive)
    idx_start = text_lower.find(start_lower)
    if idx_start == -1:
        # '### CONTEXT:' not found at all
        return False

    # Move idx_start to the end of '### CONTEXT:'
    idx_start += len(marker_start)

    # Find the end index of 'Please provide your best answer below:' (case-insensitive)
    idx_end = text_lower.find(end_lower, idx_start)

    # If the end marker isn't found, take everything after '### CONTEXT:'
    if idx_end == -1:
        context_substring = text[idx_start:].strip()
    else:
        context_substring = text[idx_start:idx_end].strip()

    # Convert the extracted substring to lowercase for checks
    context_substring_lower = context_substring.lower()

    # 1) Check if empty
    if not context_substring:
        return True

    # 2) Check if exactly '[]' (with optional whitespace inside)
    # We'll do a regex match on the trimmed substring
    # e.g. '[]', '[   ]'
    if re.fullmatch(r"\[\s*\]", context_substring.strip()):
        return True

    # 3) Check if it contains 'no relevant'
    if "no relevant" in context_substring_lower:
        return True

    # If none of the above conditions are met, return False
    return False


In [19]:
RAGv4['Context_Empty'] = RAGv4['Prompt'].apply(is_context_empty)

In [20]:
RAGv4['Context_Empty'].value_counts()

Context_Empty
True     200
False    185
Name: count, dtype: int64

In [23]:
RAGv5['Context_Empty'] = RAGv5['Prompt'].apply(is_context_empty)

In [24]:
RAGv6['Context_Empty'] = RAGv6['Prompt'].apply(is_context_empty)

In [25]:
RAGv6['Context_Empty'].value_counts()

Context_Empty
False    251
True     134
Name: count, dtype: int64

In [ ]:
# baseline_output = baseline['Generated Response'].to_list()
RAG1_output = RAGv1['Step2Response'].to_list()
RAG2_output = RAGv2['Step2Response_v2'].to_list()
RAG3_output = RAGv3['Step3Response'].to_list()
RAG4_output = RAGv4['Response'].to_list()
RAG5_output = RAGv5['Response'].to_list()
RAG6_output = RAGv6['Response'].to_list()
RAG7_output = RAGv7['Response'].to_list()

In [27]:
# baseline

In [28]:
unseen_list = RAGv3['IFUnseen'].to_list()
RAG4_unseen = RAGv4['Context_Empty'].to_list()
RAG5_unseen = RAGv5['Context_Empty'].to_list()
RAG6_unseen = RAGv6['Context_Empty'].to_list()
questions = RAGv6['Question'].to_list()
accepted_answer  = RAGv3['Accepted Answer Cleaned'].to_list()
# questions = baseline['Title'].to_list()

In [ ]:
data = pd.DataFrame({
    "Question": questions,
    "RAG1": RAG1_output,
    "RAG2": RAG2_output,
    "RAG3": RAG3_output,
    "RAG4": RAG4_output,
    "RAG5": RAG5_output,
    "RAG6": RAG6_output,
    "RAG7": RAG6_output,
    'Accepted':accepted_answer,
    "RAG1-3_unseen": unseen_list,
    "RAG4_unseen": RAG4_unseen,
    "RAG5_unseen": RAG5_unseen,
    "RAG6_unseen": RAG6_unseen,
})

In [30]:
from openai import OpenAI
import pandas as pd
import json
import time 
from tqdm import tqdm  

In [ ]:
client = OpenAI(api_key = '')

In [ ]:


# 3. Define a function that builds the prompt for the judge LLM.
def build_judge_prompt(question_title, rag3, rag2, rag1, accepted_answer):
    prompt = f"""
[QUESTION TITLE]
{question_title}
[QUESTION TITLE]

[THE START OF APPROACH 1 ANSWERS]
{rag3}
[THE END OF APPROACH 1 ANSWERS]

[THE START OF APPROACH 2 ANSWERS]
{rag2}
[THE END OF APPROACH 2 ANSWERS]

[THE START OF APPROACH 3 ANSWERS]
{rag1}
[THE END OF APPROACH 3 ANSWERS]

[THE START OF APPROACH 4 ANSWERS]
{accepted_answer}
[THE END OF APPROACH 4 ANSWERS]

"""
    return prompt.strip()


In [37]:
SYS_PROMPT = """
We would like to request your feedback on the performance of four APPROACH in response to the question in [QUESTION TITLE]
Please rate the helpfulness, accuracy, level of details of the responses. 
If you find, there is incompleted code snippet in the response, deduct the overall score.
Each approach receives an overall score on a scale of 1 to 10, where a higher score indicates better overall performance.

Please first output a single line containing only 4 values indicating the scores for APPROACH 1, 2, 3, and 4 respectively. 
The two scores are separated by a comma(,). 

"""

In [39]:

evaluations = []
scores_list = []

for idx, row in tqdm(data.iterrows(), total=len(data)):
    question_title = row["Question"]
    # accepted_answer = row["Accepted Answer Body"]
    rag3 = row["RAG3"]
    rag1 = row["RAG1"]
    rag2 = row["RAG2"]
    # rag4 = row["RAG4"]
    # rag5 = row["RAG5"]
    # baseline = row["Baseline"]
    # rag6 = row["RAG6"]
    accepted_answer  = row["Accepted"]
    # Build the prompt for your judge LLM (gpt-4o-mini)
    prompt_text = build_judge_prompt(
        question_title,
        # accepted_answer,
        rag3,
        rag2,
        rag1,
        # baseline
        accepted_answer
    )
    time.sleep(0.5)
    try:
        
        response = client.chat.completions.create(

        # response = openai.ChatCompletion.create(
            model="gpt-4o",  # Replace with your actual model name
            messages=[
                {"role": "system", "content": SYS_PROMPT},
                {"role": "user", "content": prompt_text}
            ],
            temperature=0.5  # Lower temperature for consistent evaluations
        )
        
        # # Extract the raw text the LLM returned
        # for chunk in response:
        #     print(chunk.choices[0].delta.content or "", end="")
        
        evaluation_text = response.choices[0].message.content

        print(evaluation_text)
        evaluations.append(evaluation_text)
        
    except Exception as e:
        # Handle API/network exceptions, rate limits, etc.
        print(f"OpenAI API error on row {idx}: {e}")
        evaluations.append("Error")
        scores_list.append([None, None, None, None])



  0%|          | 1/385 [00:01<07:20,  1.15s/it]

6, 7, 5, 4


  1%|          | 2/385 [00:02<06:55,  1.08s/it]

5, 7, 4, 6


  1%|          | 3/385 [00:03<06:53,  1.08s/it]

...

2, 3, 2, 1


100%|█████████▉| 384/385 [08:37<00:01,  1.35s/it]

4, 6, 8, 2


100%|██████████| 385/385 [08:38<00:00,  1.35s/it]

7, 6, 5, 4


In [40]:
filtered_evaluations = [x for x in evaluations if len(x.split(',')) == 4]
filtered_evaluations

['6, 7, 5, 4',
 '5, 7, 4, 6',
 '3, 8, 2, 1',
 '7, 8, 6, 5',
 '4, 5, 6, 5',
 '6, 5, 7, 4',
 '7, 6, 8, 7',
 '5, 6, 7, 3',
 '6, 7, 8, 5',
 '6, 7, 8, 5',
 '7, 9, 5, 3',
 '7, 8, 5, 6',
 '5, 6, 7, 3',
 '7, 6, 3, 5',
 '6, 5, 7, 3',
 '7, 3, 5, 4',
 '7, 6, 5, 4',
 '3, 7, 6, 2',
 '6, 7, 5, 2',
 '3, 6, 2, 7',
 '6, 4, 8, 2',
 '4, 6, 8, 2',
 '5, 7, 3, 2',
 '6, 7, 3, 8',
 '5, 4, 6, 5',
 '6, 6, 7, 4',
 '4, 2, 5, 3',
 '3, 4, 8, 5',
 '7, 8, 6, 7',
 '4, 8, 9, 3',
 '6, 5, 7, 8',
 '7, 6, 5, 4',
 '3, 2, 3, 1',
 '4, 2, 3, 3',
 '5, 6, 4, 3',
 '4, 6, 5, 3',
 '6, 8, 7, 4',
 '3, 7, 5, 2',
 '6, 7, 7, 2',
 '7, 8, 6, 3',
 '6, 5, 4, 3',
 '6, 8, 5, 4',
 '6, 8, 5, 6',
 '4, 6, 2, 1',
 '7, 8, 6, 5',
 '6, 5, 7, 5',
 '7, 8, 7, 5',
 '3, 4, 5, 2',
 '7, 6, 5, 7',
 '5, 7, 8, 3',
 '5, 6, 3, 4',
 '7, 6, 5, 4',
 '6, 7, 7, 4',
 '4, 5, 6, 2',
 '4, 6, 7, 2',
 '6, 8, 7, 5',
 '7, 5, 6, 4',
 '2, 3, 4, 1',
 '7, 6, 5, 4',
 '6, 7, 5, 6',
 '4, 5, 7, 3',
 '6, 7, 6, 5',
 '6, 4, 5, 3',
 '7, 6, 5, 4',
 '6, 5, 4, 2',
 '3, 6, 2, 4',
 '6, 5, 4,

In [41]:
print(len(filtered_evaluations))
print(len(evaluations))

385
385


In [43]:
# R4,R5,R3,R1,R2,R6 = map(list, zip(*[map(int, x.split(',')) for x in filtered_evaluations]))
R3,R2,R1,AC = map(list, zip(*[map(int, x.split(',')) for x in filtered_evaluations]))

In [44]:
# data['RAG4_results'] = R4
# data['RAG5_results'] = R5
data['RAG3_results'] = R3
data['RAG1_results'] = R1
data['RAG2_results'] = R2
data['AC'] = AC

In [ ]:
# data.to_csv('llm_as_judge_results/results_dataframe-new_RQ2_baseline_and_ac.csv',index=False)

In [ ]:
# data.to_csv('llm_as_judge_results/results_dataframe-2_0.9.csv',index=False)
data.to_csv('llm_as_judge_results/results_dataframe_0.1.csv',index=False)
# This data is the RAG1-RAG5 basic results
# data.to_csv('llm_as_judge_results/results_dataframe_unseen_combination.csv',index=False)

In [485]:
# This data is the RAG1-RAG5 variantion_results after discussion for additional experiments
# data.to_csv('llm_as_judge_results/all_rag_approach_variation_results.csv',index=False)

In [486]:
# This data contains RAG6 and not containing baseline
# data.to_csv('llm_as_judge_results/all_rag_approach_variation_results_with_RAG6.csv',index=False)